In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import datetime
import requests
import sched
import time
import json

s = sched.scheduler(,)

In [3]:
#抓取今天上市
res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
df = pd.read_html(res.text)[0]
# 設定column名稱
df.columns = df.iloc[0]
# 刪除第一二行
df=df.iloc[2:]
stock_dict={}
for i in df['有價證券代號及名稱']:
    if len(i.split( )) <2:
        continue
    stock_dict[i.split( )[0]]=i.split( )[1]

In [2]:
def tableColor(val):
    if val > 0:
        color = 'red'
    elif val < 0:
        color = 'green'
    else:
        color = 'white'
    return 'color: %s' % color

In [7]:
def stock_crawler(targets):
    
    #clear_output(wait=True)
    
    for key, value in stock_dict.items():
    # 組成stock_list
        stock_list = '|'.join('tse_{}.tw'.format(target) for target in targets) 

        #　query data
        query_url = "http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch="+ stock_list
        data = json.loads(urlopen(query_url).read())

        # 過濾出有用到的欄位
        columns = ['c','n','z','tv','v','o','h','l','y']
        df = pd.DataFrame(data['msgArray'], columns=columns)
        df.columns = ['股票代號','公司簡稱','當盤成交價','當盤成交量','累積成交量','開盤價','最高價','最低價','昨收價']

        # 新增漲跌百分比
        df.iloc[:, [2,3,4,5,6,7,8]] = df.iloc[:, [2,3,4,5,6,7,8]].astype(float)
        df['漲跌百分比'] = (df['當盤成交價'] - df['昨收價'])/df['昨收價'] * 100

        # 紀錄更新時間
        time = datetime.datetime.now()  
        print("更新時間:" +str(time) )

        # show table
        df = df.style.applymap(tableColor, subset=['漲跌百分比'])
        display(df)
    
    start_time = datetime.datetime.strptime(str(time.date())+'9:00', '%Y-%m-%d%H:%M')
    end_time =  datetime.datetime.strptime(str(time.date())+'13:30', '%Y-%m-%d%H:%M')
    
    # 判斷爬蟲終止條件
    if time >= start_time and time <= end_time:
        s.enter(5, 0, stock_crawler, argument=(targets,))

In [8]:
# 欲爬取的股票代碼 範圍太大有問題 需要修改


# 每秒定時器
s.enter(1, 0, stock_crawler, argument=(stock_list,))
s.run()

更新時間:2020-02-27 09:12:24.726065


,股票代號,公司簡稱,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌百分比
0,1101,台泥,42.050000,1.000000,1767.000000,42.200000,42.250000,42.050000,42.200000,-0.355450
1,1102,亞泥,44.150000,5.000000,757.000000,44.250000,44.450000,44.050000,44.300000,-0.338600
2,1103,嘉泥,20.050000,5.000000,63.000000,20.200000,20.200000,20.050000,20.100000,-0.248756
3,1104,環泥,17.900000,2.000000,38.000000,17.900000,17.900000,17.900000,17.900000,0.000000
4,1108,幸福,nan,0.000000,0.000000,nan,nan,nan,7.730000,nan
5,1109,信大,17.100000,3.000000,14.000000,17.150000,17.150000,17.100000,17.200000,-0.581395
6,1110,東泥,16.800000,173.000000,250.000000,16.850000,16.850000,16.750000,16.750000,0.298507


更新時間:2020-02-27 09:12:30.732408


,股票代號,公司簡稱,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌百分比
0,1101,台泥,42.050000,3.000000,1770.000000,42.200000,42.250000,42.050000,42.200000,-0.355450
1,1102,亞泥,44.150000,3.000000,760.000000,44.250000,44.450000,44.050000,44.300000,-0.338600
2,1103,嘉泥,20.050000,1.000000,64.000000,20.200000,20.200000,20.050000,20.100000,-0.248756
3,1104,環泥,17.900000,2.000000,38.000000,17.900000,17.900000,17.900000,17.900000,0.000000
4,1108,幸福,nan,0.000000,0.000000,nan,nan,nan,7.730000,nan
5,1109,信大,17.100000,3.000000,14.000000,17.150000,17.150000,17.100000,17.200000,-0.581395
6,1110,東泥,16.800000,173.000000,250.000000,16.850000,16.850000,16.750000,16.750000,0.298507


更新時間:2020-02-27 09:12:36.744752


,股票代號,公司簡稱,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌百分比
0,1101,台泥,42.100000,1.000000,1771.000000,42.200000,42.250000,42.050000,42.200000,-0.236967
1,1102,亞泥,44.150000,4.000000,764.000000,44.250000,44.450000,44.050000,44.300000,-0.338600
2,1103,嘉泥,20.050000,1.000000,64.000000,20.200000,20.200000,20.050000,20.100000,-0.248756
3,1104,環泥,17.900000,2.000000,38.000000,17.900000,17.900000,17.900000,17.900000,0.000000
4,1108,幸福,nan,0.000000,0.000000,nan,nan,nan,7.730000,nan
5,1109,信大,17.100000,3.000000,14.000000,17.150000,17.150000,17.100000,17.200000,-0.581395
6,1110,東泥,16.800000,173.000000,250.000000,16.850000,16.850000,16.750000,16.750000,0.298507


URLError: <urlopen error [WinError 10054] 遠端主機已強制關閉一個現存的連線。>